In [ ]:
import pandas as pd
import seaborn as sns

df = sns.load_dataset('titanic')

In [ ]:
pd.set_option('display.max_columns',15)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [ ]:
df = df.drop(['deck','embark_town'],axis=1)
df.columns.values

array(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'alive', 'alone'],
      dtype=object)

In [ ]:
rdf = df.dropna(subset=['age'],how='any',axis=0)
len(rdf)

714

In [ ]:
most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()
most_freq

'S'

In [ ]:
rdf.describe(include='all')

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,alive,alone
count,714.000000,714.000000,714,714.000000,714.000000,714.000000,714.000000,714,714,714,714,714,714
unique,NaN,NaN,2,NaN,NaN,NaN,NaN,3,3,3,2,2,2
top,NaN,NaN,male,NaN,NaN,NaN,NaN,S,Third,man,True,no,True
freq,NaN,NaN,453,NaN,NaN,NaN,NaN,556,355,413,413,424,404
mean,0.406162,2.236695,NaN,29.699118,0.512605,0.431373,34.694514,NaN,NaN,NaN,NaN,NaN,NaN
std,0.491460,0.838250,NaN,14.526497,0.929783,0.853289,52.918930,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,1.000000,NaN,0.420000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.000000,1.000000,NaN,20.125000,0.000000,0.000000,8.050000,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.000000,2.000000,NaN,28.000000,0.000000,0.000000,15.741700,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.000000,3.000000,NaN,38.000000,1.000000,1.000000,33.375000,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
rdf['embarked'].fillna(most_freq,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [ ]:
ndf = rdf[['survived','pclass','sex','age','sibsp','parch','embarked']]
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


In [ ]:
# 원핫인코딩
# 'sex'열의 'female'과 'male'값을 열 이름으로 갖는 2개의 더미 변수 열 생성
onehot_sex = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf,onehot_sex],axis=1)

# 'embarked'열의 'S','C','Q'값을 열 이름으로 갖는 3개의 더미 변수 열 생성/ 접두어 'town' 붙임
onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='town')
ndf = pd.concat([ndf,onehot_embarked],axis=1)

ndf.drop(['sex','embarked'],axis=1,inplace=True)
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


In [ ]:
ndf.columns

Index(['survived', 'pclass', 'age', 'sibsp', 'parch', 'female', 'male',
       'town_C', 'town_Q', 'town_S', 'female', 'male', 'town_C', 'town_Q',
       'town_S'],
      dtype='object')

In [ ]:
X = ndf[['pclass', 'age', 'sibsp', 'parch', 'female', 'male',
       'town_C', 'town_Q', 'town_S']]
y = ndf['survived']

In [ ]:
# 데이터 정규화
# 상대적인 크기 차이를 없애기 위하여
from sklearn import preprocessing
x = preprocessing.StandardScaler().fit(X).transform(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state = 10)

print(X_train.shape)
print(X_test.shape)

(499, 9)
(215, 9)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# 모형 객체 생성. k = 5
knn = KNeighborsClassifier(n_neighbors=5)

# 모형 학습
knn.fit(X_train, y_train)

# test 데이터를 가지고 y_hat을 예측(분류)
# 모형이 분류한 예측값을 y_hat 변수에 저장
y_hat = knn.predict(X_test)

print(y_hat[0:10])
print(y_test.values[0:10])

[0 0 1 0 0 1 0 1 0 0]
[0 0 1 0 0 1 1 1 0 0]


In [ ]:
# metrics 모듈의 confusion_matrix() 함수 사용
# 모형 성능 평가 = Confusion Matrix 계산
# 이 함수는 [[TN,FP],[FN,TP]] 형태로 반환됨.

from sklearn import metrics
knn_matrix = metrics.confusion_matrix(y_test,y_hat)
print(knn_matrix)

[[108  17]
 [ 30  60]]


In [ ]:
# 모형 성능 평가
knn_report = metrics.classification_report(y_test,y_hat)
print(knn_report)

              precision    recall  f1-score   support

           0       0.78      0.86      0.82       125
           1       0.78      0.67      0.72        90

    accuracy                           0.78       215
   macro avg       0.78      0.77      0.77       215
weighted avg       0.78      0.78      0.78       215

